In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import time
import pandas as pd


# Set up the WebDriver (using Chrome in this example)
driver_path = '../chromedriver-mac/chromedriver'  # Update this path
cService = webdriver.ChromeService(executable_path=driver_path)
driver = webdriver.Chrome(service=cService)

# Open the forum page
driver.get("https://forums.distill.io/")

# Scroll to the bottom of the page to load all posts
# Note: You may need to adjust the number of scrolls or sleep duration
for i in range(10):  # Adjust the range as needed
    driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.END)
    time.sleep(2)  # Wait for posts to load, adjust time as needed

# Now that the page is loaded, use BeautifulSoup to parse the page
soup = BeautifulSoup(driver.page_source, 'html.parser')

# Find all forum posts
posts = soup.find_all('tr', class_='topic-list-item')  # Adjust if the class name is different
posts_data = []
# Extract information from each post
i = 0
for post in posts:
    title = post.find('a', class_='title').text.strip()
    replies = post.find('td', class_='posts').text.strip()
    views = post.find('td', class_='views').text.strip()
    activity = post.find('td', class_='activity').text.strip()
    url = 'https://forums.distill.io' + post.find('a', class_='title')['href']

    posts_data.append([title, replies, views, activity, url])
    i=i+1
    if i%25==0: print(i)

# Close the browser
driver.quit()

# Create a DataFrame from the list
df = pd.DataFrame(posts_data, columns=['Title', 'Replies', 'Views', 'Activity', 'URL'])

# Optionally, save the DataFrame to a file (e.g., CSV)
df.to_csv('forum_posts.csv', index=False)

df.head()


25
50
75
100
125
150
175
200
225
250
275
300
325


,Title,Replies,Views,Activity,URL
0,Welcome to Distill.io,0,1.4k,Oct '23,https://forums.distill.io/t/welcome-to-distill...
1,How do I know my macro is working?,1,8,6h,https://forums.distill.io/t/how-do-i-know-my-m...
2,This account password cannot be initialized. P...,0,10,11h,https://forums.distill.io/t/this-account-passw...
3,No export possible using the Desktop App,2,30,15h,https://forums.distill.io/t/no-export-possible...
4,Macro based on changes?,5,43,16h,https://forums.distill.io/t/macro-based-on-cha...


In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import time

def extract_forum_posts(forum_url):
    # Initialize the WebDriver
    driver_path = '../chromedriver-mac/chromedriver'  # Update this path
    cService = webdriver.ChromeService(executable_path=driver_path)
    driver = webdriver.Chrome(service=cService)
    
    # Navigate to the forum post page
    driver.get(forum_url)
    
    # Wait for the page to load completely
    time.sleep(5)  # Adjust sleep time as needed
    
    # Parse the page with BeautifulSoup
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    
    # Close the browser
    driver.quit()
    
    # Find all posts in the page
    posts = soup.find_all('div', class_='topic-post')  # Replace with the actual class if different
    #print(posts)
    # List to hold all posts data
    posts_data = []
    
    # Extract details for each post
    for post in posts:
        author = post.find('span', class_='username').text.strip()  # Replace with the actual class if different
        dateEl = post.find('div', class_='post-date')  # Replace with the actual tag if different
        if dateEl:
            dateSpan = dateEl.find('span')
            if dateSpan and  'title' in dateSpan.attrs:
                date = dateSpan['title']
        content = post.find('div', class_='contents').text.strip()  # Replace with the actual class if different
        
        # Add the post details to the list
        posts_data.append({
            'author': author,
            'date': date,
            'content': content
        })
    
    return posts_data

# Usage:
forum_url = 'https://forums.distill.io/t/ios-app-unable-to-sync-watchlist-or-add-a-new-monitor/2850'
posts_details = extract_forum_posts(forum_url)

for post in posts_details:
    print(post)


{'author': 'porkape', 'date': 'Dec 14, 2023 6:22 am', 'content': 'First time with this service and using the latest version of the extension on Chrome with cloud monitoring for 1 product on various European Amazon sites. The iPhone app does not seem to sync with my watchlist nor can I add new monitors (only tried with Amazon links).\nHowever, when I added my phone number via the app it showed up in the extension settings on the browser so not sure why the sync is not working.\nAnd, whenever there is an error in the webapp/extension I get a notification badge on the app but the list stays empty.\nIn the extension/webapp it only shows 2 of my PCs as the devices and not my phone. On the phone app the device setting is missing.\nTo remedy I have tried to sync manually, restarted the browser + computer, tried on another PC, signed in and out of both the extension and iOS app but nothing seems to have helped.\nCould you please advise other methods I could try? Thank you'}
{'author': 'ajitk',

In [3]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import uuid

# Fetch the HTML content from a webpage
#url = 'https://forums.distill.io/t/unable-to-access-websites-when-trying-to-set-up-a-macro/2817'
#url = 'https://forums.distill.io/t/how-to-monitor-twitter-for-user-posting-a-new-tweet/998'


def extract_forum_posts_using_requestsAPI(post_url):

    response = requests.get(post_url)
    html_content = response.text
    #print(html_content)

    # Create a BeautifulSoup object
    soup = BeautifulSoup(html_content, 'html.parser')

    post_title = soup.find(id='topic-title').find('a').text.strip()
    #print(post_title)

    post_list = soup.find_all(class_='topic-body')

    author_attr = {'itemprop':'name'}
    text_attr = {'itemprop':'text'}
    date_attr = {'itemprop':'datePublished'}
    position_attr = {'itemprop':'position'}
    posts = []

    def find_atr_text(post, attrib_key_value):
        result = ''
        element = post.find(attrs=attrib_key_value)
        if element:
            result = element.text.strip()
        return result

    for post in post_list:
        author = find_atr_text(post, author_attr)
        text = find_atr_text(post, text_attr)
        date_text = find_atr_text(post, date_attr)
        position_text = find_atr_text(post, position_attr)
        combined_str = '-'.join([post_title, url, position_text])
        uniqueID = uuid.uuid5(uuid.NAMESPACE_URL, combined_str)
        posts.append({
            'uid':uniqueID,
            'title':post_title,
            'url':url,
            'author':author, 
            'content':text,
            'date':date_text,
            'order':position_text
            })
    
    return posts

forum_url = 'https://forums.distill.io/t/ios-app-unable-to-sync-watchlist-or-add-a-new-monitor/2850'
posts_details = extract_forum_posts_using_requestsAPI(forum_url)

for post in posts_details:
    print(post)




{'uid': UUID('94286b74-772f-5f56-9ede-8abeceb6536d'), 'title': 'iOS app unable to sync watchlist or add a new monitor', 'url': 'https://forums.distill.io/t/can-we-customise-the-interval-for-scheduling-our-checks-as-per-our-requirement/1821', 'author': 'porkape', 'content': 'First time with this service and using the latest version of the extension on Chrome with cloud monitoring for 1 product on various European Amazon sites. The iPhone app does not seem to sync with my watchlist nor can I add new monitors (only tried with Amazon links).\nHowever, when I added my phone number via the app it showed up in the extension settings on the browser so not sure why the sync is not working.\nAnd, whenever there is an error in the webapp/extension I get a notification badge on the app but the list stays empty.\nIn the extension/webapp it only shows 2 of my PCs as the devices and not my phone. On the phone app the device setting is missing.\nTo remedy I have tried to sync manually, restarted the b

In [4]:

numPosts = 50

forum_posts_db = pd.read_csv('forum_posts.csv')
forum_post_details = pd.DataFrame(columns = ['uid', 'title', 'url', 'author', 'content', 'date', 'order'])

for i in range(numPosts):
    row_data = forum_posts_db.iloc[i]
    url = row_data['URL']
    print(url)
    print(i)
    post_details = extract_forum_posts_using_requestsAPI(url)
    #url_df = pd.DataFrame(post_details)
    new_forum_post_details = pd.DataFrame(post_details)
    forum_post_details = pd.concat([forum_post_details, new_forum_post_details], ignore_index=True)
    
    print(30*'-')

forum_post_details.to_csv('forum_post_details.csv', index=False)
forum_post_details.head()


https://forums.distill.io/t/welcome-to-distill-io/7
0
------------------------------
https://forums.distill.io/t/how-do-i-know-my-macro-is-working/3090
1
------------------------------
https://forums.distill.io/t/this-account-password-cannot-be-initialized-please-make-it-possible-to-initialize-the-password/3088
2
------------------------------
https://forums.distill.io/t/no-export-possible-using-the-desktop-app/3083
3
------------------------------
https://forums.distill.io/t/macro-based-on-changes/3076
4
------------------------------
https://forums.distill.io/t/desktop-app-not-launching-on-mac-login/3081
5
------------------------------
https://forums.distill.io/t/how-to-setup-distill-to-track-us-embassy-visa-slots/3080
6
------------------------------
https://forums.distill.io/t/how-to-stop-monitors-running-mutliple-times-on-start-up/3085
7
------------------------------
https://forums.distill.io/t/change-notification-contents/3075
8
------------------------------
https://forums.dis

,uid,title,url,author,content,date,order
0,c897f16d-d8a3-554a-aec7-733500b736bf,Welcome to Distill.io,https://forums.distill.io/t/welcome-to-distill...,system,A community for discussing topics for web moni...,,1
1,e96b341f-ff13-5c3e-98b0-6fd41d2b2cf9,How do I know my macro is working?,https://forums.distill.io/t/how-do-i-know-my-m...,j.hapgood,I got a paid subscription so I could use a mac...,,1
2,4697d49a-8260-5429-85e7-3df5f70af338,How do I know my macro is working?,https://forums.distill.io/t/how-do-i-know-my-m...,j.hapgood,,"February 23, 2024, 5:13pm",2
3,3c484f6c-8f40-5065-9779-90639461a4ba,This account password cannot be initialized. P...,https://forums.distill.io/t/this-account-passw...,yt123190,This account password cannot be initialized. P...,,1
4,581ee545-91df-526f-bd3f-0943d85fe27c,No export possible using the Desktop App,https://forums.distill.io/t/no-export-possible...,gchampeau,"Hi there,\nThanks for your tool, it’s very han...",,1
